<a href="https://colab.research.google.com/github/acdc2019/algo-trading/blob/main/python/notebooks/strategies/rsi_strategy/RSIStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RSI Strategy**
## **15 mins strategy**
* Step 1: A 15 min candle has to close above 70 RSI value.
* Step 2: Above High of this candle Buy signal is generated.
* Step 3: Stop Loss of the trade has to be below the earlier 15 min candles low.
* Step 4: Volume of the candle in which buy signal is generated has to be atleast 2 to 3 times of earlier 5 candles (***Q: average of earlier 5 candle volume?***)
* Step 5: Hourly candles RSI should be above 50 for confirmation.
* Step 6: In the last 5 candles the stock movement should not be more than 4% of the day movement
* Step 7: If Stop Loss of the candle is less than 6000 INR then the trade should be executed

#### **Strategy Parameters**
* window_start, window_end: Dates between which to look for signal
* rsi_15min = 70
* rsi_60min = 50
* stop_loss = 6000
* back_candles = 5
* volume_multiple = 2 or 3 times
* daily_movement_pct = 4

Install libs

In [81]:
!pip install ta
!pip install pandas==1.3.5
!pip install plotly

In [91]:
!wget https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=ANF77A5O3HUPBOPUUZMLYXTBYHV2G -O helpers.py

--2021-12-21 14:57:54--  https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=ANF77A5O3HUPBOPUUZMLYXTBYHV2G
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1177 (1.1K) [text/plain]
Saving to: ‘helpers.py’

helpers.py          100%[===================>]   1.15K  --.-KB/s    in 0s      

2021-12-21 14:57:54 (51.2 MB/s) - ‘helpers.py’ saved [1177/1177]



Import Libs

In [92]:
import helpers
import pandas as pd
from ta.momentum import RSIIndicator
from plotly.subplots import make_subplots
import numpy as np

## Load 15 min and 60 min stock data and calculate RSI

In [143]:
file_15min = '/content/stock_data/HDFCBANK-HIST.csv'

df_15min = pd.read_csv(file_15min, parse_dates=['Date'], index_col=['Date'])
rsi = RSIIndicator(df_15min['Close']).rsi()
df_15min = df_15min.assign(rsi=rsi.values)
df_15min.tail()


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-17 14:15:00+05:30,1484.65,1484.70,1479.05,1481.55,151251,43.105732
2021-12-17 14:30:00+05:30,1481.65,1482.25,1474.10,1475.00,323080,36.170273
2021-12-17 14:45:00+05:30,1475.00,1475.60,1472.05,1473.85,277964,35.102404
2021-12-17 15:00:00+05:30,1473.85,1476.00,1467.90,1473.60,674159,34.861447
2021-12-17 15:15:00+05:30,1473.60,1477.90,1471.50,1476.00,646959,39.177827


## Set Strategy Parameters

In [144]:
window_start = '2021-11-01 00:00:00'
window_end = '2021-12-18 00:00:00'
rsi_15min = 70
rsi_60min = 50
lot_size = 550
stop_loss = 6000
back_candles = 5
volume_multiple = 2 # 2 or 3 times
daily_movement_pct = 4


## Actual Strategy Implementation

### Step 1: A 15 min candle has to close above 70 RSI value
Get all the candles in the window that close above required RSI value

In [145]:
curr_window_df = df_15min[window_start:window_end]

rsi_filter = (curr_window_df['rsi'] > rsi_15min)
df = curr_window_df[rsi_filter]

bullish_filter = (df['Close'] > df['Open'])
df = df[bullish_filter]

df.head()


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-11-01 14:15:00+05:30,1602.8,1609.00,1602.65,1607.80,122299,70.501552
2021-11-15 09:15:00+05:30,1562.1,1571.60,1562.10,1565.00,768939,71.953784
2021-11-15 09:30:00+05:30,1565.1,1568.50,1564.35,1568.40,200421,74.882112
2021-11-30 10:15:00+05:30,1518.3,1525.95,1516.15,1525.50,554072,71.758504
2021-11-30 10:30:00+05:30,1525.5,1529.00,1524.45,1528.05,1043120,73.105957



### Step 4: Volume check

In [146]:
temp_df = pd.DataFrame(columns=['Open','High','Low','Close','Volume','rsi','MeanVolume'])
temp_df.index.name = 'Date'
back_candles_df = pd.DataFrame()
for index, row in df.iterrows():
  # Get mean volume of previous candles
  back_candles_df = helpers.get_previous_candles(curr_window_df, index, back_candles)
  # print('Current candle:', index)
  # print('Back candles:', back_candles_df)
  mean_volume = back_candles_df['Volume'].mean()
  row['MeanVolume'] = mean_volume

  # Compare volume against previous mean volume
  if(row['Volume'] > volume_multiple*mean_volume):
    # print('Current Volume:', row['Volume'], 'greather than 2 times mean volume', mean_volume)
    temp_df.loc[index] = row

df = temp_df[['Open','High','Low','Close','Volume','rsi']]
temp_df

,Open,High,Low,Close,Volume,rsi,MeanVolume
Date,,,,,,,
2021-11-15 09:15:00+05:30,1562.1,1571.60,1562.1,1565.00,768939.0,71.953784,365589.0
2021-12-03 09:15:00+05:30,1525.8,1534.65,1525.0,1532.45,493959.0,76.965803,209737.6


### Step 5: Hourly candle RSI check

In [147]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
  df_15min_temp = df_15min[:index]
  #print(index)
  #print(df_15min_temp.tail(10))
  df_60min_o = df_15min_temp['Open'].resample('60Min', offset='30Min').apply({'Open': 'first'})
  df_60min_h = df_15min_temp['High'].resample('60Min', offset='30Min').apply({'High': 'max'})
  df_60min_l = df_15min_temp['Low'].resample('60Min', offset='30Min').apply({'Low': 'min'})
  df_60min_c = df_15min_temp['Close'].resample('60Min', offset='30Min').apply({'Close': 'last'})
  df_60min_vol = df_15min_temp['Volume'].resample('60Min', offset='30Min').apply({'Volume': 'sum'})
  df_60min = pd.concat([df_60min_o, df_60min_h, df_60min_l, df_60min_c, df_60min_vol], axis=1)
  df_60min.dropna(subset=['Open'], inplace=True)
  rsi = RSIIndicator(df_60min['Close']).rsi()
  df_60min = df_60min.assign(rsi=rsi.values)
  #print(df_60min.tail(15))
  
  rsi = df_60min.iloc[-1]['rsi']
  if(rsi > rsi_60min):
    # This 15min candle is eligible for signal
    print('Hourly candle RSI is greater than 50..', rsi)
    temp_df.loc[index] = row

df = temp_df
df


Hourly candle RSI is greater than 50.. 55.675231359367295
Hourly candle RSI is greater than 50.. 65.2533464692692


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-11-15 09:15:00+05:30,1562.1,1571.60,1562.1,1565.00,768939.0,71.953784
2021-12-03 09:15:00+05:30,1525.8,1534.65,1525.0,1532.45,493959.0,76.965803


### Step 6: Stock movement check

In [148]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
  prev_candles = helpers.get_previous_candles(df_15min, index, 5)
  min_low = prev_candles['Low'].min()
  daily_movement = row['High'] - min_low
  tanaji_pct = (daily_movement/min_low)*100
  print('daily_movement:', daily_movement,'tanaji_pct:', tanaji_pct)
  if(tanaji_pct < daily_movement_pct):
    temp_df.loc[index] = row

df = temp_df
temp_df

daily_movement: 20.949999999999818 tanaji_pct: 1.3510463354077205
daily_movement: 12.200000000000045 tanaji_pct: 0.8013399454826132


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-11-15 09:15:00+05:30,1562.1,1571.60,1562.1,1565.00,768939.0,71.953784
2021-12-03 09:15:00+05:30,1525.8,1534.65,1525.0,1532.45,493959.0,76.965803


### Step 7: Stop Loss check

In [149]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
  prev_candle = helpers.get_previous_candles(df_15min, index, 1)
  prev_low = prev_candle.iloc[0]['Low']
  if((row['High'] - prev_low)*lot_size > stop_loss):
    print('Stop Loss greater than 6000 INR. Do not trade', (row['High'] - prev_low)*lot_size)
  else:
    print('Stop Loss within range', (row['High'] - prev_low)*lot_size)
    temp_df.loc[index] = row

df = temp_df
df

Stop Loss greater than 6000 INR. Do not trade 10779.999999999949
Stop Loss within range 5857.50000000005


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-03 09:15:00+05:30,1525.8,1534.65,1525.0,1532.45,493959.0,76.965803


## **Final Signal**

In [150]:
for index, row in df.iterrows():
  prev_candle = helpers.get_previous_candles(df_15min, index, 1)
  stop_loss = prev_candle.iloc[0]['Low']
  print('For candle:', index, ', rsi:', row['rsi'], 'Entry Price', row['High'], 'Stop Loss:', stop_loss)

For candle: 2021-12-03 09:15:00+05:30 , rsi: 76.965803430323 Entry Price 1534.65 Stop Loss: 1524.0


In [151]:
import plotly.graph_objects as go
import plotly.offline as py

back=5
next=50
for index, row in df.iterrows():
  prev_candles = helpers.get_previous_candles(df_15min, index, back, True)
  next_candles = helpers.get_next_candles(df_15min, index, next)
  # print(prev_candles)
  candles = pd.concat([prev_candles, next_candles])
  # candles.loc[index] = row

  candles['DateStr'] = candles.index.strftime('%d-%m %H:%M')

  fig = make_subplots(rows=2, cols=1, shared_xaxes=False,
               vertical_spacing=0.1, subplot_titles=('OHLC', 'Volume & RSI'), 
               row_width=[0.5, 0.7],
               specs=[[{"secondary_y": False}], [{"secondary_y": True}]])
    
  fig.add_trace(go.Candlestick(x=candles['DateStr'],
                     open=candles['Open'],
                     high=candles['High'],
                     low=candles['Low'],
                     close=candles['Close'],
                     name='Signal Chart',
                     increasing_line_color='yellow',
                     increasing_fillcolor='yellow',
                     decreasing_line_color='red',
                     decreasing_fillcolor='red',),
                     row=1,col=1)
  
  fig.add_annotation(x=back,y=candles['High'][back],
                     text='Signal')
  
  # Position Entry Point
  fig.add_shape(type='line', 
                x0=-1,x1=back+next+2,
                y0=candles['High'][back], y1=candles['High'][back], 
                line=dict(color='Green'),
                row=1,col=1)
  
  # Position Stop Loss
  stop_loss_candle = helpers.get_previous_candles(df_15min, index, 1)
  fig.add_shape(type='line', 
                x0=-1,x1=back+next+2,
                y0=stop_loss_candle.iloc[0]['Low'], y1=stop_loss_candle.iloc[0]['Low'], 
                line=dict(color='Red'),
                row=1,col=1)

  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['rsi'], name='rsi',            
           marker_color='Cyan'),
           row=2, col=1)
  
  fig.add_trace(go.Bar(x=candles['DateStr'], y=candles['Volume'], name='Volume',            
           marker_color='rgb(55, 55, 109)',
           width=np.array([0.5]*candles.size)),
           secondary_y=True,
          row=2, col=1)
  
  fig.add_annotation(x=back,y=candles['rsi'][back],
                     text='Signal',row=2, col=1)
  
  fig.add_shape(type='line', x0=-1,x1=back+next+2,y0=70, y1=70, 
                line=dict(color='Green'),
                row=2,col=1)

  fig.update_xaxes(type='category', rangeslider=dict(visible=False))
  fig.update_xaxes(showgrid=False, nticks=5)
  fig.update_yaxes(showgrid=False)
  fig.update_layout(
    title='Signal generated for RSI 15mins Strategy',
    title_x = 0.5,
    autosize=False,
    width=1200,
    height=750,
    plot_bgcolor='rgb(5,5,5)',
    paper_bgcolor='rgb(0,0,0)',
    font_color='white')

  py.iplot(fig)
  